# Demo: Xsd-Parser for better understanding of standard libraries

# Loading the data and tools

In [480]:
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import csv
import xmltodict, json
import pandas as pd
from itertools import chain
import numpy as np
from tqdm import tqdm  

In [205]:
xsd_file_path = '/Users/dnm23/Projects/Score/NIEM-Releases-niem-5.1/xsd/niem-core.xsd'

In [207]:
with open(xsd_file_path, 'r') as f:
    data = f.read()
    soup = BeautifulSoup(data, "xml")

# Approach 1: Transforming xsd to dictionary

In [203]:
def print_dict_from_xsd(xsd):
    x = xmltodict.parse(xsd, process_namespaces=True)
    for key in x:
        print(json.dumps(x[key], indent=4, default=str))

In [208]:
print_dict_from_xsd(data)

{
    "@targetNamespace": "http://release.niem.gov/niem/niem-core/5.0/",
    "@version": "1",
    "@http://www.w3.org/XML/1998/namespace:lang": "en-US",
    "@http://www.w3.org/2001/XMLSchema-instance:schemaLocation": "http://release.niem.gov/niem/appinfo/5.0/ utility/appinfo.xsd http://release.niem.gov/niem/conformanceTargets/3.0/ utility/conformanceTargets.xsd",
    "@http://release.niem.gov/niem/conformanceTargets/3.0/:conformanceTargets": "http://reference.niem.gov/niem/specification/naming-and-design-rules/5.0/#ReferenceSchemaDocument",
    "@xmlns": {
        "appinfo": "http://release.niem.gov/niem/appinfo/5.0/",
        "cli": "http://reference.niem.gov/niem/specification/code-lists/5.0/code-lists-instance/",
        "ct": "http://release.niem.gov/niem/conformanceTargets/3.0/",
        "nc": "http://release.niem.gov/niem/niem-core/5.0/",
        "niem-xs": "http://release.niem.gov/niem/proxy/niem-xs/5.0/",
        "structures": "http://release.niem.gov/niem/structures/5.0/",
  

**Conclusion**: Dictionary is too long and *non-readable*. Visual tree representation would be hardly readable because of a big number of elements.

**Idea**: Decompose different types of elements and analyze them separately.

# Approach 2: Decomposing schema by different element types

## 1. Get overview of schema structure

In [368]:
all_tag_names = list(set(tag.name for tag in soup.find_all()))
all_tag_names

['complexType',
 'LocalTerm',
 'minInclusive',
 'annotation',
 'enumeration',
 'attributeGroup',
 'extension',
 'element',
 'maxExclusive',
 'sequence',
 'import',
 'minExclusive',
 'restriction',
 'appinfo',
 'complexContent',
 'list',
 'schema',
 'pattern',
 'simpleType',
 'attribute',
 'maxInclusive',
 'simpleContent',
 'documentation']

In [483]:
all_tags = set(soup.find_all())
tags_overview_df = pd.DataFrame(columns=['name', 'nested_tags', 'tag_options'])
for tag in tqdm(all_tags):
    if tag:  
        
#     1. Get nested tags
        nested_tags = [nt.name for nt in tag.findAll()]
#     todo: get nested of nested ??
        
#     2. Get tag options
        tag_str = str(tag)
        by_space = tag_str.split(' ')
        tag_options = []
        for el in by_space:
            if '=' in el:
                tag_options.append(el.split('=')[0])
                
                
        exist = tags_overview_df[tags_overview_df['name']==tag.name]
        if(exist.shape[0]==0):
            tags_overview_df = tags_overview_df.append({
                'name': tag.name,
                'nested_tags': ','.join(nested_tags),
                'tag_options': ','.join(tag_options)
            }, ignore_index=True)
        else:
            tags_overview_df.loc[tags_overview_df['name']==tag.name, 'nested_tags'] = tags_overview_df.loc[
                tags_overview_df['name']==tag.name, 'nested_tags'].iloc[0] + ',' + ','.join(nested_tags)

            
            tags_overview_df.loc[tags_overview_df['name']==tag.name, 'tag_options'] = tags_overview_df.loc[
                tags_overview_df['name']==tag.name, 'tag_options'].iloc[0] + ',' + ','.join(tag_options)

#  delete duplicated values
tags_overview_df['nested_tags'] = tags_overview_df['nested_tags'].apply(lambda value: list(set(value.split(',')))) 
tags_overview_df['tag_options'] = tags_overview_df['tag_options'].apply(lambda value: list(set(value.split(',')))) 

tags_overview_df.sort_values(by=['name'], inplace=True)
tags_overview_df

100%|██████████| 8558/8558 [00:13<00:00, 637.30it/s]


,name,nested_tags,tag_options
6,LocalTerm,[],"[term, literal, definition]"
3,annotation,"[LocalTerm, appinfo, documentation]","[, literal, term, definition]"
14,appinfo,[LocalTerm],"[term, literal, definition]"
10,attribute,"[, annotation, documentation]","[name, ref, use, type]"
21,attributeGroup,[],[ref]
11,complexContent,"[element, attribute, sequence, extension]","[ref, minOccurs, base, maxOccurs, use]"
0,complexType,"[attributeGroup, extension, element, annotatio...","[name, ref, minOccurs, base, maxOccurs, use]"
4,documentation,[],[]
1,element,"[, annotation, documentation]","[name, ref, minOccurs, abstract, type, maxOccu..."
8,enumeration,"[annotation, documentation]",[value]


In [486]:
tags_overview_df.to_csv("data/tag_overview.csv", index=False)
print('File saved at data/types.csv')

File saved at data/types.csv


In [210]:
elements = soup.findAll("xs:element")
print('--- Founded schema elements (sample):')
print(elements[:10])

--- Founded schema elements (sample):
[<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Conveyance"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityConveyanceAssociationAugmentationPoint"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Document"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityDocumentAssociationAugmentationPoint"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Facility"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityFacilityAssociationAugmentationPoint"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>]


In [248]:
complex_contents = soup.findAll("xs:complexContent")
print('--- Founded schema complexContents (sample):')
print(complex_contents[:10])

--- Founded schema complexContents (sample):
[<xs:complexContent>
<xs:extension base="nc:AssociationType">
<xs:sequence>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Conveyance"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityConveyanceAssociationAugmentationPoint"/>
</xs:sequence>
</xs:extension>
</xs:complexContent>, <xs:complexContent>
<xs:extension base="nc:AssociationType">
<xs:sequence>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Document"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityDocumentAssociationAugmentationPoint"/>
</xs:sequence>
</xs:extension>
</xs:complexContent>, <xs:complexContent>
<xs:extension base="nc:AssociationType">
<xs:sequence>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Facility"/>
<xs

## Decomposing elements

### 1. Get first overview (print every element that's missing some of the common tags)

In [235]:
elements_df = pd.DataFrame()
for xs_element in elements:
    elements_df = elements_df.append({
            'name': xs_element.get('name'),
            'type': xs_element.get('type'),
            'nillable': xs_element.get('nillable'),
            'substitutionGroup': xs_element.get('substitutionGroup'),
            'annotation_documentation': xs_element.select('annotation')[0].select('documentation')[0].text if len(xs_element.select('annotation'))>0 else None,
            'is_abstract': xs_element.get("abstract"),
            'ref': xs_element.get('ref'),  
            'minOccurs': xs_element.get('minOccurs'),
            'maxOccurs': xs_element.get('maxOccurs')
        }, ignore_index=True)
    
#     1.  Get first overview (print every element that's missing some of the common tags)
    if not xs_element.get('type'):
        print("xs:element w/o type: ", xs_element)
    if not xs_element.get('name'):
        print("xs:element w/o name: ", xs_element)
    if not xs_element.get('ref'):
        print("xs:element w/o ref: ", xs_element)

#     2. Identified scenarios based on logs:
#         a. elements with type, documentation, name, nillable (and optional substitutionGroup)
#         b. elements with abstract=true, name, documentation
#         c. elements with ref, minOccurs, maxOccurs

#     3. Print edge cases that don't fit into the identified scenarios
    if xs_element.get('name'):
        if len(xs_element.select('annotation')) == 0:
            print("name element w/o documentation: ", xs_element)
        if not xs_element.get('type') or not xs_element.get('nillable'):
            if xs_element.get("abstract") != "true":
                print("non-abstract element w/o type, nillable: ", xs_element)
        
    elif xs_element.get('ref'):
        if not xs_element.get('minOccurs') or not xs_element.get('maxOccurs'):
            print("Ref element w/o minOccurs, maxOccurs: ", xs_element)
    else:
        print("xs:element w/o name,ref: ", xs_element)
    
elements_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
elements_df.sort_values(by="name", inplace=True)
elements_df = elements_df.reindex(columns=["name", "type", "annotation_documentation", "is_abstract", "nillable", "substitutionGroup", "ref", "minOccurs", "maxOccurs"])

xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Conveyance"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Conveyance"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityConveyanceAssociationAugmentationPoint"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityConveyanceAssociationAugmentationPoint"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Document"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Document"/>
xs:element w/o type:

xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftCategoryAbstract"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftCategoryAbstract"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftFuselageColorAbstract"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftFuselageColorAbstract"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftMakeAbstract"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftMakeAbstract"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftModelAbstract"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftModelAbstract"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:AircraftStyleAbstract"/>
xs:element w/o name:  <xs:element maxOccurs="unbounde

xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryID"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryID"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryCategoryAbstract"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryCategoryAbstract"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryHash"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryHash"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryLocation"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryLocation"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryAvailableIndicator"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:BinaryAvailableIndicator"/>
xs:element w/o type:  <x

xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:CommentText"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:CommentText"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:CommentTitleText"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:CommentTitleText"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:CommentAugmentationPoint"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:CommentAugmentationPoint"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ConditionGroupID"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ConditionGroupID"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ConditionSetDate"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ConditionSetDate"/>
xs:element w/o

xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:DecalPeriodAbstract"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:DecalPeriodAbstract"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:DecalYearDate"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:DecalYearDate"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:DecalMonthDate"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:DecalMonthDate"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:DecalAugmentationPoint"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:DecalAugmentationPoint"/>
xs:element w/o type:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:RangeMinimumDecimalValue"/>
xs:element w/o name:  <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:RangeMinimumDecimalValue

...

xs:element w/o type:  <xs:element abstract="true" name="StreetAugmentationPoint">
<xs:annotation>
<xs:documentation>An augmentation point for StreetType.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element abstract="true" name="StreetAugmentationPoint">
<xs:annotation>
<xs:documentation>An augmentation point for StreetType.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o type:  <xs:element abstract="true" name="StreetCategoryAbstract">
<xs:annotation>
<xs:documentation>A data concept for a kind of street.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element abstract="true" name="StreetCategoryAbstract">
<xs:annotation>
<xs:documentation>A data concept for a kind of street.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element name="StreetCategoryText" nillable="true" substitutionGroup="nc:StreetCategoryAbstract" type="nc:TextType">
<xs:annotation>
<xs:documentation>A kind of stree

xs:element w/o ref:  <xs:element name="TaskCompletedPercent" nillable="true" type="nc:PercentType">
<xs:annotation>
<xs:documentation>A percent representing the amount of the task that has been completed.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element name="TaskDescriptionText" nillable="true" type="niem-xs:string">
<xs:annotation>
<xs:documentation>A description of the task in detail.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element name="TaskDueDate" nillable="true" type="nc:DateType">
<xs:annotation>
<xs:documentation>A date on which a task is due to be completed.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element name="TaskEndDate" nillable="true" type="nc:DateType">
<xs:annotation>
<xs:documentation>A date for the actual end of a task.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o type:  <xs:element abstract="true" name="TaskFunctionAbstract">
<xs:annotation>
<x

xs:element w/o ref:  <xs:element name="UTMCoordinateValue" nillable="true" type="nc:UTMCoordinateType">
<xs:annotation>
<xs:documentation>A coordinate from the Universal Transverse Mercator (UTM) Coordinate System, which represents a location with a hemisphere, zone, an easting value, and a northing value.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element name="Vehicle" nillable="true" type="nc:VehicleType">
<xs:annotation>
<xs:documentation>A conveyance designed to carry an operator, passengers and/or cargo, over land.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o type:  <xs:element abstract="true" name="VehicleAugmentationPoint">
<xs:annotation>
<xs:documentation>An augmentation point for VehicleType.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element abstract="true" name="VehicleAugmentationPoint">
<xs:annotation>
<xs:documentation>An augmentation point for VehicleType.</xs:documentation>
</xs:annota

xs:element w/o ref:  <xs:element name="YearDate" nillable="true" substitutionGroup="nc:DateRepresentation" type="niem-xs:gYear">
<xs:annotation>
<xs:documentation>A year.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element name="YearMonthDate" nillable="true" substitutionGroup="nc:DateRepresentation" type="niem-xs:gYearMonth">
<xs:annotation>
<xs:documentation>A year and month.</xs:documentation>
</xs:annotation>
</xs:element>
xs:element w/o ref:  <xs:element name="ZuluDateTime" nillable="true" substitutionGroup="nc:DateRepresentation" type="nc:ZuluDateTimeType">
<xs:annotation>
<xs:documentation>A date and time with no offset from Coordinated Universal Time (UTC).</xs:documentation>
</xs:annotation>
</xs:element>


,name,type,annotation_documentation,is_abstract,nillable,substitutionGroup,ref,minOccurs,maxOccurs
1303,AcquaintanceAssociation,nc:PersonAssociationType,An association between people who recognize ea...,None,true,None,None,None,None
1304,Activity,nc:ActivityType,"A single or set of related actions, events, or...",None,true,None,None,None,None
1305,ActivityActualDuration,niem-xs:duration,An actual duration of an activity.,None,true,None,None,None,None
1306,ActivityAdministratorOrganizationAssociation,nc:ActivityOrganizationAssociationType,An association between an activity and an orga...,None,true,None,None,None,None
1307,ActivityAdministratorPersonAssociation,nc:ActivityPersonAssociationType,An association between an activity and a perso...,None,true,None,None,None,None
...,...,...,...,...,...,...,...,...,...
1298,None,None,None,None,None,None,nc:WeaponInvolvedInActivity,0,unbounded
1299,None,None,None,None,None,None,nc:WeaponUsageAbstract,0,unbounded
1300,None,None,None,None,None,None,nc:WeaponAugmentationPoint,0,unbounded
1301,None,None,None,None,None,None,nc:WeightUnitAbstract,0,unbounded


In [242]:
elements_df

,name,type,annotation_documentation,is_abstract,nillable,substitutionGroup,ref,minOccurs,maxOccurs
1303,AcquaintanceAssociation,nc:PersonAssociationType,An association between people who recognize ea...,None,true,None,None,None,None
1304,Activity,nc:ActivityType,"A single or set of related actions, events, or...",None,true,None,None,None,None
1305,ActivityActualDuration,niem-xs:duration,An actual duration of an activity.,None,true,None,None,None,None
1306,ActivityAdministratorOrganizationAssociation,nc:ActivityOrganizationAssociationType,An association between an activity and an orga...,None,true,None,None,None,None
1307,ActivityAdministratorPersonAssociation,nc:ActivityPersonAssociationType,An association between an activity and a perso...,None,true,None,None,None,None
...,...,...,...,...,...,...,...,...,...
1298,None,None,None,None,None,None,nc:WeaponInvolvedInActivity,0,unbounded
1299,None,None,None,None,None,None,nc:WeaponUsageAbstract,0,unbounded
1300,None,None,None,None,None,None,nc:WeaponAugmentationPoint,0,unbounded
1301,None,None,None,None,None,None,nc:WeightUnitAbstract,0,unbounded


### 2. Identify groups based on logs

- **Group A.** elements with type, documentation, name, nillable (and optional substitutionGroup)
- **Group B.** elements with abstract=true, name, documentation
- **Group C.** elements with ref, minOccurs, maxOccurs
- *Data Types*
- *Substitution Groups*


The edge edge cases that don't fit into the identified scenarios are added to logs:

In [244]:
elements_df = pd.DataFrame()
for xs_element in elements:
    elements_df = elements_df.append({
            'name': xs_element.get('name'),
            'type': xs_element.get('type'),
            'nillable': xs_element.get('nillable'),
            'substitutionGroup': xs_element.get('substitutionGroup'),
            'annotation_documentation': xs_element.select('annotation')[0].select('documentation')[0].text if len(xs_element.select('annotation'))>0 else None,
            'is_abstract': xs_element.get("abstract"),
            'ref': xs_element.get('ref'),  
            'minOccurs': xs_element.get('minOccurs'),
            'maxOccurs': xs_element.get('maxOccurs')
        }, ignore_index=True)

#     2. Print edge cases that don't fit into the identified scenarios
    if xs_element.get('name'):
        if len(xs_element.select('annotation')) == 0:
            print("name element w/o documentation: ", xs_element)
        if not xs_element.get('type') or not xs_element.get('nillable'):
            if xs_element.get("abstract") != "true":
                print("non-abstract element w/o type, nillable: ", xs_element)
        
    elif xs_element.get('ref'):
        if not xs_element.get('minOccurs') or not xs_element.get('maxOccurs'):
            print("Ref element w/o minOccurs, maxOccurs: ", xs_element)
    else:
        print("xs:element w/o name,ref: ", xs_element)
    
elements_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
elements_df.sort_values(by="name", inplace=True)
elements_df = elements_df.reindex(columns=["name", "type", "annotation_documentation", "is_abstract", "nillable", "substitutionGroup", "ref", "minOccurs", "maxOccurs"])
elements_df

,name,type,annotation_documentation,is_abstract,nillable,substitutionGroup,ref,minOccurs,maxOccurs
1303,AcquaintanceAssociation,nc:PersonAssociationType,An association between people who recognize ea...,None,true,None,None,None,None
1304,Activity,nc:ActivityType,"A single or set of related actions, events, or...",None,true,None,None,None,None
1305,ActivityActualDuration,niem-xs:duration,An actual duration of an activity.,None,true,None,None,None,None
1306,ActivityAdministratorOrganizationAssociation,nc:ActivityOrganizationAssociationType,An association between an activity and an orga...,None,true,None,None,None,None
1307,ActivityAdministratorPersonAssociation,nc:ActivityPersonAssociationType,An association between an activity and a perso...,None,true,None,None,None,None
...,...,...,...,...,...,...,...,...,...
1298,None,None,None,None,None,None,nc:WeaponInvolvedInActivity,0,unbounded
1299,None,None,None,None,None,None,nc:WeaponUsageAbstract,0,unbounded
1300,None,None,None,None,None,None,nc:WeaponAugmentationPoint,0,unbounded
1301,None,None,None,None,None,None,nc:WeightUnitAbstract,0,unbounded


#### Group A. Elements with type, documentation, name, nillable (and optional substitutionGroup)

In [239]:
group_a = elements_df[elements_df['is_abstract'].isna() & elements_df['ref'].isna()][['name', 'type', 'nillable', 'substitutionGroup', 'annotation_documentation']]
print("Total unique elements in group a: ", group_a.shape[0])
group_a

Total unique elements in group a:  1372


,name,type,nillable,substitutionGroup,annotation_documentation
1303,AcquaintanceAssociation,nc:PersonAssociationType,true,None,An association between people who recognize ea...
1304,Activity,nc:ActivityType,true,None,"A single or set of related actions, events, or..."
1305,ActivityActualDuration,niem-xs:duration,true,None,An actual duration of an activity.
1306,ActivityAdministratorOrganizationAssociation,nc:ActivityOrganizationAssociationType,true,None,An association between an activity and an orga...
1307,ActivityAdministratorPersonAssociation,nc:ActivityPersonAssociationType,true,None,An association between an activity and a perso...
...,...,...,...,...,...
3097,WeaponUser,nc:PersonType,true,None,A person who uses a weapon.
3098,WeightMeasure,nc:WeightMeasureType,true,None,A measure of a weight.
3101,YearDate,niem-xs:gYear,true,nc:DateRepresentation,A year.
3102,YearMonthDate,niem-xs:gYearMonth,true,nc:DateRepresentation,A year and month.


#### Group B. Elements with abstract=true, name, documentation

In [240]:
group_b = elements_df[elements_df['is_abstract']=='true'][['name', 'annotation_documentation', 'is_abstract']]
print("Total unique elements in group b: ", group_b.shape[0])
group_b

Total unique elements in group b:  429


,name,annotation_documentation,is_abstract
1310,ActivityAugmentationPoint,An augmentation point for ActivityType.,true
1314,ActivityCategoryAbstract,A data concept for a kind of activity.,true
1320,ActivityConveyanceAssociationAugmentationPoint,An augmentation point for ActivityConveyanceAs...,true
1325,ActivityDocumentAssociationAugmentationPoint,An augmentation point for ActivityDocumentAsso...,true
1330,ActivityFacilityAssociationAugmentationPoint,An augmentation point for ActivityFacilityAsso...,true
...,...,...,...
3091,VolumeUnitAbstract,A data concept for a unit of measure for volume.,true
3093,WeaponAugmentationPoint,An augmentation point for WeaponType.,true
3095,WeaponUsageAbstract,A data concept for a way or manner in which a ...,true
3099,WeightMeasureAugmentationPoint,An augmentation point for type nc:WeightMeasur...,true


#### Group C. Elements with ref, minOccurs, maxOccurs

In [241]:
group_c = elements_df[elements_df['ref'].isna()==False][['ref', 'minOccurs', 'maxOccurs']]
print("Total unique elements in group c: ", group_c.shape[0])
group_c

Total unique elements in group c:  1303


,ref,minOccurs,maxOccurs
0,nc:Activity,0,unbounded
1,nc:Conveyance,0,unbounded
2,nc:ActivityConveyanceAssociationAugmentationPoint,0,unbounded
3,nc:Document,0,unbounded
4,nc:ActivityDocumentAssociationAugmentationPoint,0,unbounded
...,...,...,...
1298,nc:WeaponInvolvedInActivity,0,unbounded
1299,nc:WeaponUsageAbstract,0,unbounded
1300,nc:WeaponAugmentationPoint,0,unbounded
1301,nc:WeightUnitAbstract,0,unbounded


#### Data Types

In [237]:
unique_types = pd.DataFrame(elements_df.groupby('type')['name'].unique()).reset_index()
print("Total unique types: ", unique_types.shape[0])
unique_types

Total unique types:  254


,type,name
0,nc:ActivityConveyanceAssociationType,"[ActivityConveyanceAssociation, IncidentVehicl..."
1,nc:ActivityDocumentAssociationType,[ActivityDocumentAssociation]
2,nc:ActivityFacilityAssociationType,[ActivityFacilityAssociation]
3,nc:ActivityItemAssociationType,[ActivityItemAssociation]
4,nc:ActivityOrganizationAssociationType,"[ActivityAdministratorOrganizationAssociation,..."
...,...,...
249,niem-xs:integer,"[MeasureIntegerValue, RangeMaximumIntegerValue..."
250,niem-xs:nonNegativeInteger,"[ConveyanceEngineQuantity, FacilityCapacityQua..."
251,niem-xs:string,"[AddressDeliveryPointID, AddressGridID, Addres..."
252,niem-xs:time,"[BeginTime, EndTime, ScheduleDayEndTime, Sched..."


#### Substitution Groups

In [238]:
unique_substitution_groups = pd.DataFrame(elements_df.groupby('substitutionGroup')['name'].unique()).reset_index()
unique_substitution_groups.columns = ['substitutionGroup', 'element_names']
print("Total unique substitution groups: ", unique_substitution_groups.shape[0])
unique_substitution_groups

Total unique substitution groups:  202


,substitutionGroup,element_names
0,nc:ActivityCategoryAbstract,[ActivityCategoryText]
1,nc:ActivityDisposition,[CaseDisposition]
2,nc:ActivityInvolvementAbstract,[ActivityInvolvementText]
3,nc:ActivityReasonAbstract,[ActivityReasonText]
4,nc:ActivityRelationAbstract,[ActivityRelationText]
...,...,...
197,nc:VehicleTransmissionCategoryAbstract,[VehicleTransmissionCategoryText]
198,nc:VesselHullMaterialAbstract,[VesselHullMaterialText]
199,nc:VesselHullShapeAbstract,[VesselHullShapeText]
200,nc:VesselPropulsionAbstract,[VesselPropulsionText]


## Decomposing complex types

In [249]:
complex_contents[:5]

[<xs:complexContent>
 <xs:extension base="nc:AssociationType">
 <xs:sequence>
 <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
 <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Conveyance"/>
 <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityConveyanceAssociationAugmentationPoint"/>
 </xs:sequence>
 </xs:extension>
 </xs:complexContent>,
 <xs:complexContent>
 <xs:extension base="nc:AssociationType">
 <xs:sequence>
 <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
 <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Document"/>
 <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityDocumentAssociationAugmentationPoint"/>
 </xs:sequence>
 </xs:extension>
 </xs:complexContent>,
 <xs:complexContent>
 <xs:extension base="nc:AssociationType">
 <xs:sequence>
 <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
 <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Facility"/>
 <xs:element maxOccurs="un

In [302]:
complex_contents_df = pd.DataFrame()
for content in complex_contents:
    refs = [el.get('ref') for el in content.select('extension')[0].select('sequence')[0].select('element')]
    complex_contents_df = complex_contents_df.append({
                'extension_base': content.select('extension')[0].get('base'),
                'refs': ','.join(refs),
                'ref_count': len(refs)
            }, ignore_index=True)
    
    if content.select("extension") == None or len(content.select("extension")) != 1:
        print("complex content with extension (len=: ", len(content.select("extension")), "): ", content)
    elif len(content.select('extension')[0].select('sequence')) < 1:
        print("complex content with extension sequence len<1: ", content)
        
complex_contents_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
complex_contents_df.sort_values(by="extension_base", inplace=True)

In [303]:
complex_contents_df

,extension_base,ref_count,refs
179,nc:ActivityType,3.0,"nc:SystemOperatingModeAbstract,nc:SystemUserNa..."
153,nc:ActivityType,14.0,"nc:ProgramPerson,nc:ProgramAction,nc:ProgramCa..."
154,nc:ActivityType,5.0,"nc:ProgressComplianceIndicator,nc:ProgressComp..."
137,nc:ActivityType,10.0,"nc:PersonArmedIndicator,nc:PersonAssaultedOffi..."
33,nc:ActivityType,8.0,"nc:ConditionGroupID,nc:ConditionSetDate,nc:Con..."
...,...,...,...
115,structures:ObjectType,14.0,"nc:MessageID,nc:MessageRecipientAbstract,nc:Me..."
114,structures:ObjectType,8.0,"nc:MedicalConditionDoctor,nc:MedicalConditionT..."
113,structures:ObjectType,10.0,"nc:MeasureValueAbstract,nc:MeasureCommentText,..."
126,structures:ObjectType,39.0,"nc:OrganizationAbbreviationText,nc:Organizatio..."


In [314]:
extension_base_types = complex_contents_df.extension_base.value_counts()
print("Total unique  extension base types: ", extension_base_types.shape[0])
print(extension_base_types)

Total unique  extension base types:  23
structures:ObjectType                    101
nc:AssociationType                        37
nc:ActivityType                           16
nc:MeasureType                            11
nc:ItemType                                8
nc:ConveyanceType                          3
nc:EntityType                              2
nc:DocumentType                            2
nc:DispositionType                         2
nc:PersonLocationAssociationType           1
structures:AssociationType                 1
nc:Location2DGeospatialCoordinateType      1
nc:SpeedMeasureType                        1
nc:EmploymentAssociationType               1
nc:FacilityType                            1
structures:MetadataType                    1
nc:SubstanceType                           1
nc:RelatedActivityAssociationType          1
nc:IntangibleItemType                      1
nc:PersonAssociationType                   1
nc:IdentificationType                      1
nc:LengthMeasur

In [309]:
print("Total unique complex contents: ", complex_contents_df.shape[0])
complex_contents_df

Total unique complex contents:  196


,extension_base,ref_count,refs
179,nc:ActivityType,3.0,"nc:SystemOperatingModeAbstract,nc:SystemUserNa..."
153,nc:ActivityType,14.0,"nc:ProgramPerson,nc:ProgramAction,nc:ProgramCa..."
154,nc:ActivityType,5.0,"nc:ProgressComplianceIndicator,nc:ProgressComp..."
137,nc:ActivityType,10.0,"nc:PersonArmedIndicator,nc:PersonAssaultedOffi..."
33,nc:ActivityType,8.0,"nc:ConditionGroupID,nc:ConditionSetDate,nc:Con..."
...,...,...,...
115,structures:ObjectType,14.0,"nc:MessageID,nc:MessageRecipientAbstract,nc:Me..."
114,structures:ObjectType,8.0,"nc:MedicalConditionDoctor,nc:MedicalConditionT..."
113,structures:ObjectType,10.0,"nc:MeasureValueAbstract,nc:MeasureCommentText,..."
126,structures:ObjectType,39.0,"nc:OrganizationAbbreviationText,nc:Organizatio..."


# Final Report

In [226]:
def extract_elements_df(elements):
    elements_df = pd.DataFrame()
    for xs_element in elements:
        elements_df = elements_df.append({
                'name': xs_element.get('name'),
                'type': xs_element.get('type'),
                'nillable': xs_element.get('nillable'),
                'substitutionGroup': xs_element.get('substitutionGroup'),
                'annotation_documentation': xs_element.select('annotation')[0].select('documentation')[0].text if len(xs_element.select('annotation'))>0 else None,
                'is_abstract': xs_element.get("abstract"),
                'ref': xs_element.get('ref'),  
                'minOccurs': xs_element.get('minOccurs'),
                'maxOccurs': xs_element.get('maxOccurs')
            }, ignore_index=True)

    #     1.  Get first overview (print every element that's missing some of the common tags)
    #     if not xs_element.get('type'):
    #         print("xs:element w/o type: ", xs_element)
    #     if not xs_element.get('name'):
    #         print("xs:element w/o name: ", xs_element)
    #     if not xs_element.get('ref'):
    #         print("xs:element w/o ref: ", xs_element)

    #     2. Identified scenarios based on logs:
    #         a. elements with type, documentation, name, nillable (and optional substitutionGroup)
    #         b. elements with abstract=true, name, documentation
    #         c. elements with ref, minOccurs, maxOccurs

    #     3. Print edge cases that don't fit into the identified scenarios
        if xs_element.get('name'):
            if len(xs_element.select('annotation')) == 0:
                print("name element w/o documentation: ", xs_element)
            if not xs_element.get('type') or not xs_element.get('nillable'):
                if xs_element.get("abstract") != "true":
                    print("non-abstract element w/o type, nillable: ", xs_element)

        elif xs_element.get('ref'):
            if not xs_element.get('minOccurs') or not xs_element.get('maxOccurs'):
                print("Ref element w/o minOccurs, maxOccurs: ", xs_element)
        else:
            print("xs:element w/o name,ref: ", xs_element)

    elements_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
    elements_df.sort_values(by="name", inplace=True)
    elements_df = elements_df.reindex(columns=["name", "type", "annotation_documentation", "is_abstract", "nillable", "substitutionGroup", "ref", "minOccurs", "maxOccurs"])
    return elements_df


In [316]:
def extract_complex_contents(complex_contents):
    complex_contents_df = pd.DataFrame()
    for content in complex_contents:
        refs = [el.get('ref') for el in content.select('extension')[0].select('sequence')[0].select('element')]
        complex_contents_df = complex_contents_df.append({
                    'extension_base': content.select('extension')[0].get('base'),
                    'refs': ','.join(refs),
                    'ref_count': len(refs)
                }, ignore_index=True)

        if content.select("extension") == None or len(content.select("extension")) != 1:
            print("complex content with extension (len=: ", len(content.select("extension")), "): ", content)
        elif len(content.select('extension')[0].select('sequence')) < 1:
            print("complex content with extension sequence len<1: ", content)

    complex_contents_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
    complex_contents_df.sort_values(by="extension_base", inplace=True)
    return complex_contents_df

In [315]:
def create_report(elements_df, complex_contents_df):

    print('--------------------------------------------------------------------')
    print('Report:')

    unique_types = pd.DataFrame(elements_df.groupby('type')['name'].unique()).reset_index()
    print("Total unique types: ", unique_types.shape[0])
    print("Sample: ")
    print(unique_types.iloc[:10, :])
    unique_types.to_csv('data/types.csv', index=False)
    print('File saved at data/types.csv')

    print('--------------------------------------------------------------------')
    unique_substitution_groups = pd.DataFrame(elements_df.groupby('substitutionGroup')['name'].unique()).reset_index()
    unique_substitution_groups.columns = ['substitutionGroup', 'element_names']
    print("Total unique substitution groups: ", unique_substitution_groups.shape[0])
    print("Sample: ")
    print(unique_substitution_groups.iloc[:10, :])
    unique_substitution_groups.to_csv('data/substitution_groups.csv', index=False)
    print('File saved at data/substitution_groups.csv')

    print('--------------------------------------------------------------------')
    print("Group a: elements with type, documentation, name, nillable (and optional substitutionGroup)")

    group_a = elements_df[elements_df['is_abstract'].isna() & elements_df['ref'].isna()][['name', 'type', 'nillable', 'substitutionGroup', 'annotation_documentation']]

    print("Total unique elements in group a: ", group_a.shape[0])
    print("Sample: ")
    print(group_a.iloc[:10, :])
    group_a.to_csv('data/a_elements_with_name_type.csv', index=False)
    print('File saved at data/a_elements_with_name_type.csv')


    print('--------------------------------------------------------------------')
    print("Group b: elements with abstract=true, name, documentation")

    group_b = elements_df[elements_df['is_abstract']=='true'][['name', 'annotation_documentation', 'is_abstract']]

    print("Total unique elements in group b: ", group_b.shape[0])
    print("Sample: ")
    print(group_b.iloc[:10, :])
    group_b.to_csv('data/abstract_elements.csv', index=False)
    print('File saved at data/abstract_elements.csv')


    print('--------------------------------------------------------------------')
    print("Group c: elements with ref, minOccurs, maxOccurs")

    group_c = elements_df[elements_df['ref'].isna()==False][['ref', 'minOccurs', 'maxOccurs']]

    print("Total unique elements in group c: ", group_c.shape[0])
    print("Sample: ")
    print(group_c.iloc[:10, :])
    group_c.to_csv('data/ref_elements.csv', index=False)
    print('File saved at data/ref_elements.csv')
    
    print('--------------------------------------------------------------------')
    print("Extension base types for complex contents:")

    extension_base_types = complex_contents_df.extension_base.value_counts()
    
    print("Total unique  extension base types: ", extension_base_types.shape[0])
    print(extension_base_types)
    extension_base_types.to_csv('data/extension_base_types.csv', index=False)
    print('File saved at data/extension_base_types.csv')
    
    print('--------------------------------------------------------------------')
    print("Group d: complex contents:")
    
    print("Total unique complex contents: ", complex_contents_df.shape[0])
    print("Sample: ")
    print(complex_contents_df.iloc[:10, :])
    complex_contents_df.to_csv('data/complex_contents.csv', index=False)
    print('File saved at data/complex_contents.csv')
    

In [318]:
# 1. Loading data
xsd_file_path = '/Users/dnm23/Projects/Score/NIEM-Releases-niem-5.1/xsd/niem-core.xsd'
with open(xsd_file_path, 'r') as f:
    data = f.read()
    soup = BeautifulSoup(data, "xml")
elements = soup.findAll("xs:element")

# Preview
print('--- Founded schema elements (sample):')
print(elements[:3])
complex_contents = soup.findAll("xs:complexContent")
print('--- Founded schema complexContents (sample):')
print(complex_contents[:3])

# 3. Processing
elements_df = extract_elements_df(elements)
complex_contents_df = extract_complex_contents(complex_contents)

# 4. Report
create_report(elements_df, complex_contents_df)

--- Founded schema elements (sample):
[<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Conveyance"/>, <xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityConveyanceAssociationAugmentationPoint"/>]
--- Founded schema complexContents (sample):
[<xs:complexContent>
<xs:extension base="nc:AssociationType">
<xs:sequence>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Conveyance"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityConveyanceAssociationAugmentationPoint"/>
</xs:sequence>
</xs:extension>
</xs:complexContent>, <xs:complexContent>
<xs:extension base="nc:AssociationType">
<xs:sequence>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Activity"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:Document"/>
<xs:element maxOccurs="unbounded" minOccurs="0" ref="nc:ActivityDocumentAssociationAu